This notebook takes the raw data found in multiple files within the "raw" folder and places them all into one file for a given month. The data is separated into motor vehicle theft and theft and finally geocoded so that it can be used in the map construction.

In [1]:
import pandas as pd
import os
import geocoder
import requests

In [2]:
for folder in os.listdir('./raw'):
    file_list = []
    for file in os.listdir('./raw/' + folder):
        if(file.endswith('.xls')):
            data = pd.read_excel('./raw/' + folder + '/' + file)
            file_list.append(data)
   
    df = pd.concat(file_list, ignore_index=True)
    
    columns = ['Incident Number','Date', 'Time', 'Police District','Offense 1',
          'Offense 2', 'Offense 3', 'Offense 4', 'Offense 5', 'Location']
    df.columns = columns
    
    df = df[df['Offense 1'].notnull()]
    df = df[df['Location'].notnull()]
    
    df.to_csv('./raw/' + folder + 'Complete.csv', sep=';', index = False)

In [3]:
def address_to_latlong(address):
    '''Returns the latitude and longitude for a given address'''
    address += ", Milwaukee, WI"
    location = geocoder.arcgis(address, session = session)
    
    if not location.latlng:
        print("Couldn't Parse: "  + address )
    else:
        print(f'Parsed: {address} - {location.latlng}')
    return location.latlng

In [4]:
for file in os.listdir('./raw/'):
    if(file.endswith('.csv')):
        df = pd.read_csv('./raw/'+ file, sep =';')
    
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
              (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
              (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
              (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
              (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                df = df.drop(off_1[0])
 
        df['Address'] = df['Location'].copy()
            
        theft = df.copy()
        m_theft = df.copy()
        
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'motor vehicle theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
                (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
                (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
                (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
                (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                m_theft = m_theft.drop(off_1[0])
            else:
                theft = theft.drop(off_1[0])
                
        
        with requests.Session() as session:
            # This is likely to be very slow
            # depending on the geocoding service used
            theft['Location'] = theft['Location'].apply(address_to_latlong)
            m_theft['Location'] = m_theft['Location'].apply(address_to_latlong)
            
        theft.to_csv('./' + 'theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        m_theft.to_csv('./' + 'm_theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        print(f'Finished {file}')

Parsed: 4356 N 16TH ST, Milwaukee, WI - [43.09622080099834, -87.93049291185314]
Parsed: 5000 N 53RD ST, Milwaukee, WI - [43.10939929063965, -87.97874533741185]
Parsed: 4035 N 15TH ST, Milwaukee, WI - [43.0902997543713, -87.92952162479058]
Parsed: 4411 N 27TH ST, Milwaukee, WI - [43.09765609447046, -87.94682809525831]
Parsed: 7110 N 39TH ST, Milwaukee, WI - [43.14685365889551, -87.95988241185313]
Parsed: 3120 W THURSTON AV, Milwaukee, WI - [43.120959525967905, -87.95199247446477]
Parsed: 4957 N 36TH ST, Milwaukee, WI - [43.10764300582838, -87.95777907372015]
Parsed: 6925 N 37TH ST, Milwaukee, WI - [43.143346937442274, -87.95703033136904]
Parsed: 2228 W LANCASTER AV, Milwaukee, WI - [43.110182903124944, -87.9394958614761]
Parsed: 4851 N 39TH ST #A, Milwaukee, WI - [43.10573483819033, -87.96156110646872]
Parsed: 2032 W EGGERT PL, Milwaukee, WI - [43.11086132444748, -87.93826790235582]
Parsed: 6038 N 39TH ST, Milwaukee, WI - [43.12757599417162, -87.96090586078269]
Parsed: 2830 W BOBOLINK A

Parsed: 317 N 92ND ST, Milwaukee, WI - [43.03446300000002, -88.02751760646872]
Parsed: 6233 W LOCUST ST, Milwaukee, WI - [43.07165753952926, -87.9906735]
Parsed: 6205 W BURLEIGH ST, Milwaukee, WI - [43.075337502885546, -87.98994]
Parsed: 2323 N 55TH ST, Milwaukee, WI - [43.06112999999999, -87.98196157703836]
Parsed: 1632 N 57TH ST, Milwaukee, WI - [43.0522652514571, -87.98416488963613]
Parsed: 5615 W MARTIN DR, Milwaukee, WI - [43.04605094751592, -87.98354689246341]
Parsed: 5500 W MARTIN DR, Milwaukee, WI - [43.04604319081398, -87.98235019081397]
Parsed: 6246 W LISBON AV, Milwaukee, WI - [43.07009461645206, -87.99111632970754]
Parsed: 2719 N 54TH ST, Milwaukee, WI - [43.068482863725535, -87.9810110809335]
Parsed: 1334 N 45TH ST, Milwaukee, WI - [43.048277664723884, -87.97022386799604]
Parsed: 5000 W CHAMBERS ST, Milwaukee, WI - [43.073474483841586, -87.9761812590745]
Parsed: 3518 N 63RD ST, Milwaukee, WI - [43.0831887171793, -87.99087542627986]
Parsed: 333 N 35TH ST, Milwaukee, WI - [4

Parsed: 2515 S 9TH PL, Milwaukee, WI - [42.998823922009365, -87.92371954818492]
Parsed: 1801 S 18TH ST, Milwaukee, WI - [43.01125708775813, -87.93540136565447]
Parsed: 1500 W NATIONAL AV, Milwaukee, WI - [43.02316550432783, -87.93064305981302]
Parsed: 1909 W WHITE ST, Milwaukee, WI - [43.01283406380022, -87.93754868779038]
Parsed: 1245-A S 15TH ST, Milwaukee, WI - [43.018049064112176, -87.930640092042]
Parsed: 300 W NATIONAL AV, Milwaukee, WI - [43.02331816196853, -87.91408966196852]
Parsed: 1514 S COMSTOCK AV, Milwaukee, WI - [43.01530429557397, -87.93767879320805]
Parsed: 2059 S 16TH ST, Milwaukee, WI - [43.00706770191587, -87.93327897879261]
Parsed: 201 W NATIONAL AV, Milwaukee, WI - [43.0231618380315, -87.91272166196852]
Parsed: 151 W PIERCE ST, Milwaukee, WI - [43.024097836844064, -87.9119429354458]
Parsed: 2035 S 7TH ST, Milwaukee, WI - [43.00750875437129, -87.920038092042]
Parsed: 700 S 3RD ST, Milwaukee, WI - [43.024135575310275, -87.91393723918766]
Parsed: 300 W NATIONAL AV, M

Parsed: 3520 S 15TH ST, Milwaukee, WI - [42.98054400000001, -87.93157494460172]
Parsed: 4624 S 27TH ST, Milwaukee, WI - [42.960725717179315, -87.94850297287826]
Parsed: 5541 S 13TH ST, Milwaukee, WI - [42.943760863725544, -87.92947751543632]
Parsed: 5114 S 27TH ST, Milwaukee, WI - [42.951815968636396, -87.94870495455638]
Parsed: 5934 S HOWELL AV, Milwaukee, WI - [42.936688363841114, -87.90959446497426]
Parsed: 1606 W WILBUR AV, Milwaukee, WI - [42.97750810392281, -87.93406472385064]
Parsed: 3116 S 15TH PL, Milwaukee, WI - [42.987779580904856, -87.93263597013694]
Parsed: 1700 W HOLT AV, Milwaukee, WI - [42.98300769406257, -87.93502751952431]
Parsed: 4570 S LENOX ST, Milwaukee, WI - [42.96120258090485, -87.89835497013695]
Parsed: 4454 S 14TH ST, Milwaukee, WI - [42.963525000000004, -87.93044596292359]
Parsed: 4100 S 27TH ST, Milwaukee, WI - [42.970275555369625, -87.94834545946208]
Parsed: 2371 S 16TH ST, Milwaukee, WI - [43.0013705029142, -87.93341005929342]
Parsed: 2906 S 20TH ST, Milwa

Parsed: 2819 N 39TH ST, Milwaukee, WI - [43.07018400000001, -87.96217760646873]
Parsed: 1400 W CHERRY ST, Milwaukee, WI - [43.050048195901134, -87.93012319590115]
Parsed: 4222 W GARFIELD AV, Milwaukee, WI - [43.059259507646054, -87.966432]
Parsed: 2535 N 15TH ST, Milwaukee, WI - [43.064856167638055, -87.93134914311246]
Parsed: 1011 W CENTER ST, Milwaukee, WI - [43.06756145557012, -87.92422595458292]
Parsed: 2326 N 47TH ST, Milwaukee, WI - [43.061246748542914, -87.97243291185313]
Parsed: 3203 W NORTH AV, Milwaukee, WI - [43.06059152120739, -87.95400316763806]
Parsed: 2463 N 39TH ST, Milwaukee, WI - [43.063831005828376, -87.96226760646873]
Parsed: 2428 N 48TH ST, Milwaukee, WI - [43.06312766472388, -87.97370491906649]
Parsed: 1834 W WALNUT ST, Milwaukee, WI - [43.052712504327836, -87.93619272300768]
Parsed: 1950 N 20TH ST, Milwaukee, WI - [43.05597654094289, -87.93765392627985]
Parsed: 2618 N 47TH ST, Milwaukee, WI - [43.06656697446479, -87.97236638631792]
Parsed: 2728 N GRANT BL, Milwau

...

Parsed: 8300 N 93RD ST, Milwaukee, WI - [43.168902770938004, -88.02761070348538]
Parsed: 6924 N RAINTREE DR, Milwaukee, WI - [43.14324757995415, -87.98243214600785]
Parsed: 8001 W BEECHWOOD AV, Milwaukee, WI - [43.14229951399403, -88.01071216763806]
Parsed: 6960 N LINCOLNSHIRE CR, Milwaukee, WI - [43.14460768348519, -87.9812879257908]
Parsed: 6419 N 89TH ST, Milwaukee, WI - [43.1347142514571, -88.0229881392173]
Parsed: 6550 N 76TH ST, Milwaukee, WI - [43.13694042576028, -88.00556237621996]
Parsed: 7501 N GRANVILLE RD, Milwaukee, WI - [43.15441060342088, -88.02873145510482]
Parsed: 7710 W DEAN RD, Milwaukee, WI - [43.17056947489748, -88.00570319608748]
Parsed: 8950 N 95TH ST, Milwaukee, WI - [43.178970363033066, -88.02860219357731]
Parsed: 8838 W ACACIA ST, Milwaukee, WI - [43.13789549321934, -88.02246205536962]
Parsed: 6965 N 55TH ST, Milwaukee, WI - [43.1441660641122, -87.98060910156303]
Parsed: 10507 W ARCH AV, Milwaukee, WI - [43.16650389757064, -88.04228728838928]
Parsed: 7813 N 60

Parsed: 829 S 19TH ST, Milwaukee, WI - [43.02204227699232, -87.93681956650677]
Parsed: 1028 W PIERCE ST, Milwaukee, WI - [43.02425249321934, -87.92485183236194]
Parsed: 2489 S 6TH ST, Milwaukee, WI - [42.999489670552265, -87.91889152596791]
Parsed: 500 W LAPHAM BL, Milwaukee, WI - [43.01434295654488, -87.91679623578683]
Parsed: 315 S WATER ST, Milwaukee, WI - [43.0283580047032, -87.90709838138358]
Parsed: 2235 S 11TH ST, Milwaukee, WI - [43.003810257285465, -87.92587000822297]
Parsed: 1515 W NATIONAL AV, Milwaukee, WI - [43.02308252842077, -87.93110947293557]
Parsed: 1523 S 6TH ST, Milwaukee, WI - [43.01543745045876, -87.91846151543632]
Parsed: 809 S BARCLAY ST, Milwaukee, WI - [43.02290828864909, -87.90984799711447]
Parsed: 1310 W GREENFIELD AV, Milwaukee, WI - [43.017053529863055, -87.92837983236194]
Parsed: 1422 S 9TH ST, Milwaukee, WI - [43.016488491257434, -87.9225254662418]
Parsed: 2324 S 11TH ST, Milwaukee, WI - [43.00224232944774, -87.92599494460173]
Parsed: 1400 S 20TH ST, Mil

Parsed: 5706 N 92ND ST, Milwaukee, WI - [43.12152807799066, -88.02705391185313]
Parsed: 6618 W MEDFORD AV, Milwaukee, WI - [43.10302675894762, -87.99457322367822]
Parsed: 4062 N 61ST ST, Milwaukee, WI - [43.09144153300744, -87.98813168277537]
Parsed: 8726 W CARMEN AV, Milwaukee, WI - [43.12349750432784, -88.02120998167814]
Parsed: 8000 W BENDER AV, Milwaukee, WI - [43.13233048600597, -88.01094060935425]
Parsed: 5265 N 83RD ST, Milwaukee, WI - [43.11367200000001, -88.01461911368207]
Parsed: 4871 N 65TH ST, Milwaukee, WI - [43.10640836081134, -87.99257060646872]
Parsed: 5427 N 91ST ST, Milwaukee, WI - [43.116426251457085, -88.02603816475253]
Parsed: 7824 W VILLARD AV, Milwaukee, WI - [43.112272493219315, -88.00917308381904]
Parsed: 5364 N 62ND ST, Milwaukee, WI - [43.11524915820299, -87.98860282705316]
Parsed: 5040 N 65TH ST, Milwaukee, WI - [43.10927138773155, -87.99245388963614]
Parsed: 5845 N 83RD ST #4, Milwaukee, WI - [43.12422058673323, -88.01453765753918]
Parsed: 6029 W MARION ST,

Parsed: 2747 N 2ND ST, Milwaukee, WI - [43.06851016763807, -87.91256060646873]
Parsed: 2408 N 4TH ST, Milwaukee, WI - [43.06228183236195, -87.91551589684948]
Parsed: 1006 W NASH ST, Milwaukee, WI - [43.08445349321932, -87.92380047737896]
Parsed: 3389 N 20TH ST, Milwaukee, WI - [43.08081294754459, -87.93727813200395]
Parsed: 824 W NASH ST, Milwaukee, WI - [43.08444248211083, -87.92173274562872]
Parsed: 3227 N 15TH ST, Milwaukee, WI - [43.0774200313636, -87.92986812479059]
Parsed: 3642 N 25TH ST, Milwaukee, WI - [43.08456630391251, -87.94474237910455]
Parsed: 1800 W ATKINSON AV, Milwaukee, WI - [43.08833462663833, -87.93329081127618]
Parsed: 1500 W BURLEIGH ST, Milwaukee, WI - [43.07499150764604, -87.93010266180968]
Parsed: 3367 N 3RD ST, Milwaukee, WI - [43.08031875437129, -87.913810092042]
Parsed: 3165 N 11TH ST, Milwaukee, WI - [43.076295502914206, -87.92498863200395]
Parsed: 2477 N 2ND ST, Milwaukee, WI - [43.06375816763807, -87.91261761368207]
Parsed: 1429 N 6TH ST, Milwaukee, WI - 

Parsed: 9626 W BRADLEY RD, Milwaukee, WI - [43.16316645383429, -88.03086948098658]
Parsed: 6643 N 52ND ST, Milwaukee, WI - [43.13866533527613, -87.97701966475253]
Parsed: 8975 W BROWN DEER RD, Milwaukee, WI - [43.177607531738985, -88.0230102485429]
Parsed: 8601 W DENVER AV, Milwaukee, WI - [43.1402325212074, -88.01926303664372]
Parsed: 7323 N 86TH ST, Milwaukee, WI - [43.150832660081974, -88.018266333571]
Parsed: 8901 W PORTAGE ST, Milwaukee, WI - [43.15527205040441, -88.02305220138062]
Parsed: 7006 N 86TH ST, Milwaukee, WI - [43.145116910352584, -88.0178828496742]
Parsed: 6945 N 76TH ST #103, Milwaukee, WI - [43.14482306411219, -88.00565065422094]
Parsed: 7278 N 86TH ST, Milwaukee, WI - [43.15024729808414, -88.01819341185313]
Parsed: 6244 W PORT AV, Milwaukee, WI - [43.158424011945215, -87.98828419954992]
Parsed: 6570 N 80TH ST, Milwaukee, WI - [43.13757592809752, -88.01052232630335]
Parsed: 5000 W VILLARD AV, Milwaukee, WI - [43.11199452986306, -87.97544091618097]
Parsed: 7027 N 85TH